# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint
# Import API key
from api_keys import geoapify_key


In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,23.51,78,100,5.92,ZA,1703920624
1,1,waitangi,-43.9535,-176.5597,17.68,96,100,3.13,NZ,1703920624
2,2,walvis bay,-22.9575,14.5053,17.83,75,0,1.45,NaN,1703920625
3,3,ushuaia,-54.8000,-68.3000,7.81,87,75,11.83,AR,1703920625
4,4,labytnangi,66.6572,66.4183,-15.64,78,50,10.53,RU,1703920626


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.75,
    color="City"
)
    

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
# Temperature criteria: 21 < Max Temp < 27
# Wind speed criteria: Wind Speed < 4.5m/s
# Cloudiness criteria: Cloudiness = 0

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
cit = data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,srirampur,22.9489,88.0189,26.49,37,0,3.02,IN,1703920630
34,34,madhubani,26.3667,86.0833,24.43,39,0,2.72,IN,1703920643
147,147,memari,23.2000,88.1167,26.32,37,0,2.99,IN,1703920704
189,189,lashio,22.9333,97.7500,22.97,44,0,1.63,MM,1703920725
333,333,kintampo,8.0563,-1.7306,21.87,25,0,0.69,GH,1703920797


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,srirampur,IN,22.9489,88.0189,37,
34,madhubani,IN,26.3667,86.0833,39,
147,memari,IN,23.2000,88.1167,37,
189,lashio,MM,22.9333,97.7500,44,
333,kintampo,GH,8.0563,-1.7306,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}
   
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
srirampur - nearest hotel: No hotel found
madhubani - nearest hotel: Madhuyamini
memari - nearest hotel: No hotel found
lashio - nearest hotel: Golden Kaniri Hotel
kintampo - nearest hotel: No hotel found
maraba - nearest hotel: Portobelo
vila velha - nearest hotel: Hotel Prainha
chabahar - nearest hotel: هتل سپیده
san pedro pochutla - nearest hotel: Olimpia
gogrial - nearest hotel: No hotel found
irymple - nearest hotel: Chaffey Motor Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
12,srirampur,IN,22.9489,88.0189,37,No hotel found
34,madhubani,IN,26.3667,86.0833,39,Madhuyamini
147,memari,IN,23.2000,88.1167,37,No hotel found
189,lashio,MM,22.9333,97.7500,44,Golden Kaniri Hotel
333,kintampo,GH,8.0563,-1.7306,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=0.75,
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)